In [1]:
"""
MSBA 503 Take-Home Assignment - Object Detection
Jupyter Notebook Version - Ready for Teacher Testing

Author: Taylor Piecukonis
Date: December 2025

SETUP INSTRUCTIONS:
1. Run: !pip install tensorflow tensorflow-hub opencv-python numpy pillow pandas matplotlib
2. Create folder named 'images' in same directory as this notebook
3. Add your images to the 'images' folder
4. Run all cells in order
"""

import sys
!{sys.executable} -m pip install tensorflow tensorflow-hub opencv-python pillow pandas matplotlib

# ============================================================================
# CELL 1: INSTALL AND IMPORT LIBRARIES
# ============================================================================

# Uncomment the line below to install required packages in Jupyter
# !pip install tensorflow tensorflow-hub opencv-python numpy pillow pandas matplotlib

import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import time
import os
from PIL import Image
import pandas as pd
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("MSBA 503 - OBJECT DETECTION ASSIGNMENT")
print("="*80)
print(f"\nTensorFlow Version: {tf.__version__}")
print(f"OpenCV Version: {cv2.__version__}")
print(f"NumPy Version: {np.__version__}")
print("\n✓ All libraries imported successfully!\n")

# ============================================================================
# CELL 2: CONFIGURATION
# ============================================================================

# Configuration settings
IMAGE_FOLDER = 'images'  # Folder containing your images
CONFIDENCE_THRESHOLD = 0.3  # Minimum confidence for detections (30%)
MAX_DISPLAY_OBJECTS = 5  # Max objects to display per image

print("Configuration:")
print(f"  Image Folder: {IMAGE_FOLDER}")
print(f"  Confidence Threshold: {CONFIDENCE_THRESHOLD}")
print(f"  Max Display Objects: {MAX_DISPLAY_OBJECTS}\n")

# ============================================================================
# CELL 3: COCO CLASS NAMES
# ============================================================================

# COCO dataset class names (80 object classes)
COCO_CLASSES = {
    1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane',
    6: 'bus', 7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light',
    11: 'fire hydrant', 13: 'stop sign', 14: 'parking meter', 15: 'bench',
    16: 'bird', 17: 'cat', 18: 'dog', 19: 'horse', 20: 'sheep',
    21: 'cow', 22: 'elephant', 23: 'bear', 24: 'zebra', 25: 'giraffe',
    27: 'backpack', 28: 'umbrella', 31: 'handbag', 32: 'tie', 33: 'suitcase',
    34: 'frisbee', 35: 'skis', 36: 'snowboard', 37: 'sports ball', 38: 'kite',
    39: 'baseball bat', 40: 'baseball glove', 41: 'skateboard', 42: 'surfboard',
    43: 'tennis racket', 44: 'bottle', 46: 'wine glass', 47: 'cup',
    48: 'fork', 49: 'knife', 50: 'spoon', 51: 'bowl', 52: 'banana',
    53: 'apple', 54: 'sandwich', 55: 'orange', 56: 'broccoli', 57: 'carrot',
    58: 'hot dog', 59: 'pizza', 60: 'donut', 61: 'cake', 62: 'chair',
    63: 'couch', 64: 'potted plant', 65: 'bed', 67: 'dining table',
    70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse', 75: 'remote',
    76: 'keyboard', 77: 'cell phone', 78: 'microwave', 79: 'oven',
    80: 'toaster', 81: 'sink', 82: 'refrigerator', 84: 'book',
    85: 'clock', 86: 'vase', 87: 'scissors', 88: 'teddy bear',
    89: 'hair drier', 90: 'toothbrush'
}

def get_class_name(class_id):
    """Convert class ID to readable name"""
    return COCO_CLASSES.get(class_id, f'unknown_class_{class_id}')

print(f"✓ Loaded {len(COCO_CLASSES)} COCO object classes\n")

# ============================================================================
# CELL 4: LOAD DEEP LEARNING MODELS (PART A-i)
# ============================================================================

print("="*80)
print("LOADING DEEP LEARNING MODELS")
print("="*80)
print("\nThis will download models on first run (~2-3 minutes)")
print("Subsequent runs will be instant!\n")

# Model 1: SSD MobileNet V2 (Fast and efficient)
print("Loading Model 1: SSD MobileNet V2...")
try:
    model1 = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")
    model1_name = "SSD MobileNet V2"
    print("✓ SSD MobileNet V2 loaded successfully!")
except Exception as e:
    print(f"✗ Error loading Model 1: {e}")
    model1 = None

print()

# Model 2: EfficientDet D0 (More accurate)
print("Loading Model 2: EfficientDet D0...")
try:
    model2 = hub.load("https://tfhub.dev/tensorflow/efficientdet/d0/1")
    model2_name = "EfficientDet D0"
    print("✓ EfficientDet D0 loaded successfully!")
except Exception as e:
    print(f"✗ Error loading Model 2: {e}")
    model2 = None

print("\n✓ Both models loaded successfully!\n")

# ============================================================================
# CELL 5: OBJECT DETECTION FUNCTION (PART A-i)
# ============================================================================

def detect_objects(model, model_name, image_path, image_name):
    """
    Perform object detection on an image
    
    Args:
        model: TensorFlow Hub detection model
        model_name: Name of the model
        image_path: Path to image file
        image_name: Name for display
        
    Returns:
        dict: Detection results
    """
    try:
        # Load and preprocess image
        img = cv2.imread(image_path)
        if img is None:
            print(f"  ✗ Error: Could not load {image_path}")
            return None
        
        # Convert BGR to RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Convert to tensor
        input_tensor = tf.convert_to_tensor(img_rgb)
        input_tensor = input_tensor[tf.newaxis, ...]
        
        # Run detection and measure time
        start_time = time.time()
        detections = model(input_tensor)
        detection_time = time.time() - start_time
        
        # Extract results
        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                     for key, value in detections.items()}
        
        detection_classes = detections['detection_classes'].astype(np.int64)
        detection_scores = detections['detection_scores']
        
        # Filter by confidence threshold
        detected_objects = []
        for i in range(num_detections):
            if detection_scores[i] >= CONFIDENCE_THRESHOLD:
                detected_objects.append({
                    'class_id': int(detection_classes[i]),
                    'class_name': get_class_name(detection_classes[i]),
                    'confidence': float(detection_scores[i])
                })
        
        # Calculate average confidence
        avg_confidence = np.mean([obj['confidence'] for obj in detected_objects]) if detected_objects else 0
        
        return {
            'image': image_name,
            'model': model_name,
            'detection_time_seconds': round(detection_time, 4),
            'num_objects_detected': len(detected_objects),
            'average_confidence': round(avg_confidence, 4),
            'detected_objects': detected_objects
        }
        
    except Exception as e:
        print(f"  ✗ Error processing {image_name}: {e}")
        return None

print("✓ Object detection function defined\n")

# ============================================================================
# CELL 6: ADDITIONAL FEATURE EXTRACTION (PART A-ii)
# ============================================================================

def extract_additional_features(image_path, image_name):
    """
    Extract additional features using computer vision techniques
    
    Args:
        image_path: Path to image file
        image_name: Name for display
        
    Returns:
        dict: Extracted features
    """
    try:
        # Load image
        img = cv2.imread(image_path)
        if img is None:
            return None
        
        # Convert to different color spaces
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        
        # 1. Image dimensions
        height, width = img.shape[:2]
        dimensions = f"{width}x{height}"
        
        # 2. Face detection (Haar Cascade)
        face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        faces = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(30, 30))
        num_faces = len(faces)
        
        # 3. Eye detection (Haar Cascade)
        eye_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_eye.xml'
        )
        eyes = eye_cascade.detectMultiScale(gray, 1.1, 5, minSize=(20, 20))
        num_eyes = len(eyes)
        
        # 4. Edge detection (Canny)
        edges = cv2.Canny(gray, 100, 200)
        edge_density = (np.sum(edges > 0) / edges.size) * 100
        
        # 5. Brightness (HSV V channel)
        brightness = np.mean(hsv[:, :, 2])
        
        # 6. Contrast (standard deviation)
        contrast = np.std(gray)
        
        # 7. Dominant color (average RGB)
        avg_r = int(np.mean(img_rgb[:, :, 0]))
        avg_g = int(np.mean(img_rgb[:, :, 1]))
        avg_b = int(np.mean(img_rgb[:, :, 2]))
        dominant_color = f"({avg_r}, {avg_g}, {avg_b})"
        
        # 8. Color variance
        color_variance = np.var(img_rgb)
        
        # 9. Corner detection (Harris)
        gray_float = np.float32(gray)
        dst = cv2.cornerHarris(gray_float, 2, 3, 0.04)
        num_corners = int(np.sum(dst > 0.01 * dst.max()))
        
        # 10. Blur detection (Laplacian variance)
        blur_score = cv2.Laplacian(gray, cv2.CV_64F).var()
        
        return {
            'image': image_name,
            'dimensions': dimensions,
            'num_faces': num_faces,
            'num_eyes': num_eyes,
            'edge_density_percent': round(edge_density, 2),
            'brightness_level': round(brightness, 2),
            'contrast_level': round(contrast, 2),
            'dominant_color_rgb': dominant_color,
            'color_variance': round(color_variance, 2),
            'num_corners': num_corners,
            'blur_score': round(blur_score, 2)
        }
        
    except Exception as e:
        print(f"  ✗ Error extracting features from {image_name}: {e}")
        return None

print("✓ Feature extraction function defined\n")

# ============================================================================
# CELL 7: GET IMAGES FROM FOLDER
# ============================================================================

# Check if image folder exists
if not os.path.exists(IMAGE_FOLDER):
    os.makedirs(IMAGE_FOLDER)
    print(f"✓ Created folder: {IMAGE_FOLDER}")
    print(f"\n⚠ Please add at least 10 images to the '{IMAGE_FOLDER}' folder")
    print("   Supported formats: .jpg, .jpeg, .png, .bmp, .tiff, .webp\n")
    image_files = []
else:
    # Get all image files
    extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp')
    image_files = [f for f in os.listdir(IMAGE_FOLDER) 
                  if f.lower().endswith(extensions)]
    image_files = sorted(image_files)
    
    print(f"✓ Found {len(image_files)} images in '{IMAGE_FOLDER}' folder")
    
    if len(image_files) < 10:
        print(f"⚠ Warning: Assignment requires at least 10 images. You have {len(image_files)}.")
    
    if image_files:
        print("\nImages found:")
        for i, img in enumerate(image_files[:5], 1):
            print(f"  {i}. {img}")
        if len(image_files) > 5:
            print(f"  ... and {len(image_files) - 5} more")
    print()

# ============================================================================
# CELL 8: PROCESS ALL IMAGES
# ============================================================================

if not image_files:
    print("⚠ No images to process. Please add images and run again.")
else:
    print("="*80)
    print(f"PROCESSING {len(image_files)} IMAGES")
    print("="*80)
    print()
    
    # Store all results
    all_detection_results = []
    all_feature_results = []
    
    # Process each image
    for idx, image_file in enumerate(image_files, 1):
        image_path = os.path.join(IMAGE_FOLDER, image_file)
        image_name = os.path.splitext(image_file)[0]
        
        print(f"[{idx}/{len(image_files)}] Processing: {image_name}")
        print("-" * 60)
        
        # PART A(i): Object Detection with both models
        print("  Running object detection...")
        
        if model1:
            result1 = detect_objects(model1, model1_name, image_path, image_name)
            if result1:
                objects_list = [obj['class_name'] for obj in result1['detected_objects'][:MAX_DISPLAY_OBJECTS]]
                print(f"    {result1['model']:<20} | "
                      f"Time: {result1['detection_time_seconds']:.4f}s | "
                      f"Objects: {result1['num_objects_detected']:2} | "
                      f"Conf: {result1['average_confidence']:.4f}")
                if objects_list:
                    print(f"      → {', '.join(objects_list)}")
                all_detection_results.append(result1)
        
        if model2:
            result2 = detect_objects(model2, model2_name, image_path, image_name)
            if result2:
                objects_list = [obj['class_name'] for obj in result2['detected_objects'][:MAX_DISPLAY_OBJECTS]]
                print(f"    {result2['model']:<20} | "
                      f"Time: {result2['detection_time_seconds']:.4f}s | "
                      f"Objects: {result2['num_objects_detected']:2} | "
                      f"Conf: {result2['average_confidence']:.4f}")
                if objects_list:
                    print(f"      → {', '.join(objects_list)}")
                all_detection_results.append(result2)
        
        # PART A(ii): Additional Feature Extraction
        print("\n  Extracting additional features...")
        features = extract_additional_features(image_path, image_name)
        
        if features:
            print(f"    • Dimensions: {features['dimensions']}")
            print(f"    • Faces: {features['num_faces']}, Eyes: {features['num_eyes']}")
            print(f"    • Edge Density: {features['edge_density_percent']}%")
            print(f"    • Brightness: {features['brightness_level']}, "
                  f"Contrast: {features['contrast_level']}")
            print(f"    • Dominant Color: RGB{features['dominant_color_rgb']}")
            print(f"    • Corners: {features['num_corners']}, "
                  f"Blur: {features['blur_score']}")
            all_feature_results.append(features)
        
        print()
    
    print("="*80)
    print("✓ PROCESSING COMPLETE!")
    print("="*80)
    print()

# ============================================================================
# CELL 9: GENERATE SUMMARY STATISTICS
# ============================================================================

if all_detection_results:
    print("="*80)
    print("SUMMARY STATISTICS")
    print("="*80)
    print()
    
    # Group by model
    model1_results = [r for r in all_detection_results if r['model'] == model1_name]
    model2_results = [r for r in all_detection_results if r['model'] == model2_name]
    
    # Calculate statistics for Model 1
    if model1_results:
        avg_time_1 = np.mean([r['detection_time_seconds'] for r in model1_results])
        avg_objects_1 = np.mean([r['num_objects_detected'] for r in model1_results])
        avg_conf_1 = np.mean([r['average_confidence'] for r in model1_results if r['average_confidence'] > 0])
        
        print(f"{model1_name}:")
        print(f"  Average Detection Time:    {avg_time_1:.4f} seconds")
        print(f"  Average Objects Detected:  {avg_objects_1:.2f} objects")
        print(f"  Average Confidence Score:  {avg_conf_1:.4f}")
        print()
    
    # Calculate statistics for Model 2
    if model2_results:
        avg_time_2 = np.mean([r['detection_time_seconds'] for r in model2_results])
        avg_objects_2 = np.mean([r['num_objects_detected'] for r in model2_results])
        avg_conf_2 = np.mean([r['average_confidence'] for r in model2_results if r['average_confidence'] > 0])
        
        print(f"{model2_name}:")
        print(f"  Average Detection Time:    {avg_time_2:.4f} seconds")
        print(f"  Average Objects Detected:  {avg_objects_2:.2f} objects")
        print(f"  Average Confidence Score:  {avg_conf_2:.4f}")
        print()
    
    # Performance comparison
    if model1_results and model2_results:
        speed_diff = ((avg_time_2 - avg_time_1) / avg_time_1) * 100
        objects_diff = ((avg_objects_2 - avg_objects_1) / avg_objects_1) * 100
        conf_diff = ((avg_conf_2 - avg_conf_1) / avg_conf_1) * 100
        
        print("Performance Comparison:")
        print(f"  {model2_name} is {abs(speed_diff):.1f}% {'slower' if speed_diff > 0 else 'faster'} than {model1_name}")
        print(f"  {model2_name} detects {abs(objects_diff):.1f}% {'more' if objects_diff > 0 else 'fewer'} objects")
        print(f"  {model2_name} has {abs(conf_diff):.1f}% {'higher' if conf_diff > 0 else 'lower'} confidence")
        print()

# ============================================================================
# CELL 10: CREATE DATAFRAMES AND EXPORT TO CSV
# ============================================================================

if all_detection_results:
    print("="*80)
    print("CREATING OUTPUT FILES")
    print("="*80)
    print()
    
    # PART A(i): Detection Comparison Table
    detection_data = []
    for result in all_detection_results:
        # Get detected objects list
        objects = ', '.join([obj['class_name'] for obj in result['detected_objects'][:5]])
        if len(result['detected_objects']) > 5:
            objects += f" (+{len(result['detected_objects'])-5} more)"
        
        detection_data.append({
            'Image Name': result['image'],
            'Model': result['model'],
            'Detection Time (s)': result['detection_time_seconds'],
            'Objects Detected': result['num_objects_detected'],
            'Avg Confidence': result['average_confidence'],
            'Detected Objects': objects if objects else 'None'
        })
    
    df_detection = pd.DataFrame(detection_data)
    df_detection.to_csv('part_a_i_detection_comparison.csv', index=False)
    print("✓ Part A(i) saved to: part_a_i_detection_comparison.csv")
    
    # Display detection table
    print("\nPart A(i) - Detection Comparison Table:")
    print(df_detection.to_string(index=False))
    print()
    
    # PART A(ii): Additional Features Table
    if all_feature_results:
        df_features = pd.DataFrame(all_feature_results)
        df_features.to_csv('part_a_ii_additional_features.csv', index=False)
        print("✓ Part A(ii) saved to: part_a_ii_additional_features.csv")
        
        # Display features table
        print("\nPart A(ii) - Additional Features Table:")
        print(df_features.to_string(index=False))
        print()
    
    # Save detailed JSON
    output_json = {
        'metadata': {
            'date_processed': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'total_images': len(image_files),
            'models_used': [model1_name, model2_name]
        },
        'detection_results': all_detection_results,
        'feature_results': all_feature_results
    }
    
    with open('detailed_results.json', 'w') as f:
        json.dump(output_json, f, indent=2)
    print("✓ Detailed results saved to: detailed_results.json")
    
    print("\n" + "="*80)
    print("✓ ALL FILES GENERATED SUCCESSFULLY!")
    print("="*80)
    print("\nGenerated files:")
    print("  1. part_a_i_detection_comparison.csv  - For Part A(i) in Word")
    print("  2. part_a_ii_additional_features.csv  - For Part A(ii) in Word")
    print("  3. detailed_results.json              - Full detailed results")
    print("\nYou can now copy these tables into your Word document!")
    print("="*80)


MSBA 503 - OBJECT DETECTION ASSIGNMENT

TensorFlow Version: 2.20.0
OpenCV Version: 4.12.0
NumPy Version: 2.1.3

✓ All libraries imported successfully!

Configuration:
  Image Folder: images
  Confidence Threshold: 0.3
  Max Display Objects: 5

✓ Loaded 80 COCO object classes

LOADING DEEP LEARNING MODELS

This will download models on first run (~2-3 minutes)
Subsequent runs will be instant!

Loading Model 1: SSD MobileNet V2...



✓ SSD MobileNet V2 loaded successfully!

Loading Model 2: EfficientDet D0...


✓ EfficientDet D0 loaded successfully!

✓ Both models loaded successfully!

✓ Object detection function defined

✓ Feature extraction function defined

✓ Found 11 images in 'images' folder

Images found:
  1. 1000_F_187616108_v8oMYk5q7rKJXtZOfWOLoz51HcVu0JZl.jpg
  2. 360_F_310422728_gvGj4cxa8MZDbJOPws9MgQXUslt7jpxF.jpg
  3. DSC6580-684x1024.jpg
  4. coconuts-on-beach-place-milk-260nw-1026353836.webp
  5. empty-classroom-scene-with-interior-decoration-and-objects-free-vector.jpg
  ... and 6 more

PROCESSING 11 IMAGES

[1/11] Processing: 1000_F_187616108_v8oMYk5q7rKJXtZOfWOLoz51HcVu0JZl
------------------------------------------------------------
  Running object detection...
    SSD MobileNet V2     | Time: 5.6999s | Objects: 100 | Conf: 0.3788
      → chair, chair, chair, chair, chair
    EfficientDet D0      | Time: 10.1763s | Objects: 11 | Conf: 0.4580
      → chair, potted plant, laptop, chair, chair

  Extracting additional features...
    • Dimensions: 1000x714
    • Faces: 0, Eye

In [ ]:
Part A(i) - Detection Comparison Table:
                                                            Image Name            Model  Detection Time (s)  Objects Detected  Avg Confidence                                                  Detected Objects
                     1000_F_187616108_v8oMYk5q7rKJXtZOfWOLoz51HcVu0JZl SSD MobileNet V2              5.6999               100          0.3788                      chair, chair, chair, chair, chair (+95 more)
                     1000_F_187616108_v8oMYk5q7rKJXtZOfWOLoz51HcVu0JZl  EfficientDet D0             10.1763                11          0.4580               chair, potted plant, laptop, chair, chair (+6 more)
                      360_F_310422728_gvGj4cxa8MZDbJOPws9MgQXUslt7jpxF SSD MobileNet V2              0.8915                74          0.4128                       car, car, car, car, potted plant (+69 more)
                      360_F_310422728_gvGj4cxa8MZDbJOPws9MgQXUslt7jpxF  EfficientDet D0              2.6096                10          0.5646               car, car, car, potted plant, potted plant (+5 more)
                                                      DSC6580-684x1024 SSD MobileNet V2              0.7558                35          0.4198           potted plant, potted plant, dog, bowl, chair (+30 more)
                                                      DSC6580-684x1024  EfficientDet D0              2.6005                 8          0.5657               cat, cat, cat, potted plant, potted plant (+3 more)
                         coconuts-on-beach-place-milk-260nw-1026353836 SSD MobileNet V2              0.7036                14          0.4158          bed, teddy bear, teddy bear, donut, teddy bear (+9 more)
                         coconuts-on-beach-place-milk-260nw-1026353836  EfficientDet D0              2.3278                 3          0.3458                                       teddy bear, teddy bear, bed
empty-classroom-scene-with-interior-decoration-and-objects-free-vector SSD MobileNet V2              1.1956               100          0.4037         chair, chair, book, dining table, dining table (+95 more)
empty-classroom-scene-with-interior-decoration-and-objects-free-vector  EfficientDet D0              2.9359                16          0.3740 chair, dining table, dining table, chair, dining table (+11 more)
                    img-A-Guide-to-Choosing-the-Best-Toys-for-Your-Dog SSD MobileNet V2              0.8946                 3          0.5016                                            dog, sports ball, kite
                    img-A-Guide-to-Choosing-the-Best-Toys-for-Your-Dog  EfficientDet D0              2.4215                 2          0.8860                                                  dog, sports ball
                                        istockphoto-1755563909-612x612 SSD MobileNet V2              0.2863                54          0.4135                        oven, oven, knife, knife, spoon (+49 more)
                                        istockphoto-1755563909-612x612  EfficientDet D0              0.9452                 3          0.5187                                        oven, potted plant, bottle
                                         istockphoto-495977454-612x612 SSD MobileNet V2              0.3590                 3          0.5988                                            person, person, person
                                         istockphoto-495977454-612x612  EfficientDet D0              0.8601                 3          0.3900                                           person, frisbee, person
                                        pexels-ash-craig-122861-376464 SSD MobileNet V2              0.2880                20          0.4161                cake, bowl, dining table, cup, sandwich (+15 more)
                                        pexels-ash-craig-122861-376464  EfficientDet D0              0.8415                 4          0.5513                                   cake, bowl, dining table, knife
                                     pexels-ella-olsson-572949-1640777 SSD MobileNet V2              0.3438               100          0.4202                   bowl, carrot, carrot, carrot, carrot (+95 more)
                                     pexels-ella-olsson-572949-1640777  EfficientDet D0              0.8612                18          0.4209           fork, carrot, dining table, broccoli, carrot (+13 more)
                                      remove-distracting-objects_step1 SSD MobileNet V2              0.3181                16          0.3785         umbrella, person, person, person, potted plant (+11 more)
                                      remove-distracting-objects_step1  EfficientDet D0              0.8692                 4          0.4588                             boat, surfboard, surfboard, surfboard


In [ ]:
#Across the 11 images, SSD MobileNet V2 was much faster and detected far more objects on average than EfficientDet D0, but with slightly lower 
#confidence scores. EfficientDet D0 produced fewer detections overall, yet its average confidence was about 16% higher, suggesting it is more 
#conservative but surer about the objects it does detect. Overall, SSD MobileNet V2 is better for speed and coverage, while EfficientDet D0 
#trades speed and count for higher-confidence predictions.

In [ ]:
Part A(ii) - Additional Features Table:
                                                                 image dimensions  num_faces  num_eyes  edge_density_percent  brightness_level  contrast_level dominant_color_rgb  color_variance  num_corners  blur_score
                     1000_F_187616108_v8oMYk5q7rKJXtZOfWOLoz51HcVu0JZl   1000x714          0         5                  5.95            172.85           53.84    (172, 167, 161)         2963.32         5751     1041.08
                      360_F_310422728_gvGj4cxa8MZDbJOPws9MgQXUslt7jpxF    540x360          1         1                 22.49            156.40           67.44    (136, 121, 112)         6085.49         7775     5026.74
                                                      DSC6580-684x1024   684x1024          0         0                 10.33            147.36           66.49    (141, 129, 116)         4776.75         8662      693.64
                         coconuts-on-beach-place-milk-260nw-1026353836    390x280          0         1                 11.09            197.02           61.57    (160, 176, 158)         5647.67         1717     2339.14
empty-classroom-scene-with-interior-decoration-and-objects-free-vector   1736x980          0         0                  6.15            174.16           59.41    (143, 145, 110)         5123.11         3867      446.22
                    img-A-Guide-to-Choosing-the-Best-Toys-for-Your-Dog   1024x688          1         8                  2.38            237.50           41.14    (233, 211, 115)         4581.20          555      169.48
                                        istockphoto-1755563909-612x612    612x408          0         1                  7.67            157.67           63.89    (156, 153, 144)         4220.36         1980      868.57
                                         istockphoto-495977454-612x612    612x408          0         0                 11.76            188.30           71.69    (185, 167, 120)         7537.81        13263     1784.68
                                        pexels-ash-craig-122861-376464  2841x1735          1        27                  2.07            143.41           67.05    (134, 125, 114)         5103.85         9673      143.42
                                     pexels-ella-olsson-572949-1640777  4288x2859          7        38                  8.05            203.29           62.77    (197, 182, 162)         5391.76       114717      792.97
                                      remove-distracting-objects_step1  1800x1200          1         0                  7.89            174.98           49.63    (130, 143, 148)         3881.24        10878      615.78

✓ Detailed results saved to: detailed_results.json

In [ ]:
#In addition to object detection, I extracted non–deep-learning features such as image dimensions, number of faces/eyes, edge density, 
#brightness, contrast, dominant color, number of corners, and a blur score. These metrics summarize overall image complexity and quality, for example, 
#busy city and food images show higher edge density, corners, and contrast than the simpler lake and toy images. Such features could be used alongside 
#detection outputs to understand how image structure and clarity influence model performance.